# Example of using the PyRadiomics toolbox in Python

First, import some built-in Python modules needed to get our testing data.
Second, import the toolbox, only the `featureextractor` is needed, this module handles the interaction with other parts of the toolbox.

In [1]:
from __future__ import print_function, unicode_literals, division, absolute_import
import os  # needed navigate the system to get the input data
from radiomics import featureextractor  # This module is used for interaction with pyradiomics

### Setting up data

Before we can extract features, we need to get the input data, define the parameters for the extraction and instantiate the class contained within `featureextractor`.

In the next cell we get our testing data, this consists of an image and corresponding segmentation. This is also available from the PyRadiomics repository and is stored in `\pyradiomics\data`, whereas this file (and therefore, the current directory) is `\pyradiomics\bin\Notebooks`

In [2]:
# Define the testcase name
testCase = 'brain1'

# Get the relative path to pyradiomics\data
# os.cwd() returns the current working directory
# ".." points to the parent directory: \pyradiomics\bin\Notebooks\..\ is equal to \pyradiomics\bin\
# Move up 2 directories (i.e. go to \pyradiomics\) and then move into \pyradiomics\data
dataDir = os.path.join(os.getcwd(), "..", "..", "data")
print("dataDir, relative path:", dataDir)
print("dataDir, absolute path:", os.path.abspath(dataDir))

# Store the file paths of our testing image and label map into two variables
imagePath = os.path.join(dataDir, testCase + "_image.nrrd")
labelPath = os.path.join(dataDir, testCase + "_label.nrrd")

# Additonally, store the location of the example parameter file, stored in \pyradiomics\bin
paramPath = os.path.join(os.getcwd(), "..", "Params.yaml")
print("Parameter file, absolute path:", os.path.abspath(paramPath))

dataDir, relative path: /Users/mader/Dropbox/Informatics/pyradiomics/bin/Notebooks/../../data
dataDir, absolute path: /Users/mader/Dropbox/Informatics/pyradiomics/data
Parameter file, absolute path: /Users/mader/Dropbox/Informatics/pyradiomics/bin/Params.yaml


### Instantiating the extractor

Now that we have our input, we need to define the parameters and instantiate the extractor.
For this there are three possibilities:

1. Use defaults, don't define custom settings

2. Define parameters in a dictionary, control filters and features after initialisation

3. Use a parameter file

##### Method 1, use defaults

In [3]:
# Instantiate the extractor
extractor = featureextractor.RadiomicsFeaturesExtractor()

print("Extraction parameters:\n\t", extractor.kwargs)
print("Enabled filters:\n\t", extractor.inputImages)
print("Enabled features:\n\t", extractor.enabledFeatures)

Extraction parameters:
	 {'resampledPixelSpacing': None, 'interpolator': 3, 'label': 1, 'verbose': False, 'padDistance': 5}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'glcm': [], 'glrlm': [], 'glszm': [], 'shape': [], 'firstorder': []}


##### Method 2, hard-coded settings:

In [4]:
# First define the parameters
params = {}
params['binWidth'] = 20
params['sigma'] = [1, 2, 3]
params['verbose'] = True

# Instantiate the extractor
extractor = featureextractor.RadiomicsFeaturesExtractor(**params)  # ** 'unpacks' the dictionary in the function call

print("Extraction parameters:\n\t", extractor.kwargs)
print("Enabled filters:\n\t", extractor.inputImages)  # Still the default settings
print("Enabled features:\n\t", extractor.enabledFeatures)  # Still the default settings

Extraction parameters:
	 {'verbose': True, 'sigma': [1, 2, 3], 'binWidth': 20, 'resampledPixelSpacing': None, 'interpolator': 3, 'padDistance': 5, 'label': 1}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'glcm': [], 'glrlm': [], 'glszm': [], 'shape': [], 'firstorder': []}


In [5]:
# This cell is equivalent to the previous cell
extractor = featureextractor.RadiomicsFeaturesExtractor(binWidth=20, sigma=[1, 2, 3], verbose=True)  # Equivalent of code above

print("Extraction parameters:\n\t", extractor.kwargs)
print("Enabled filters:\n\t", extractor.inputImages)  # Still the default settings
print("Enabled features:\n\t", extractor.enabledFeatures)  # Still the default settings

Extraction parameters:
	 {'verbose': True, 'sigma': [1, 2, 3], 'binWidth': 20, 'resampledPixelSpacing': None, 'interpolator': 3, 'padDistance': 5, 'label': 1}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'glcm': [], 'glrlm': [], 'glszm': [], 'shape': [], 'firstorder': []}


In [6]:
# Enable a filter (in addition to the 'Original' filter already enabled)
extractor.enableInputImageByName('LoG')
print("")
print("Enabled filters:\n\t", extractor.inputImages)

# Disable all feature classes, save firstorder
extractor.disableAllFeatures()
extractor.enableFeatureClassByName('firstorder')
print("")
print("Enabled features:\n\t", extractor.enabledFeatures)

# Specify some additional features in the GLCM feature class
extractor.enableFeaturesByName(glcm=['Autocorrelation', 'Homogeneity1', 'SumSquares'])
print("")
print("Enabled features:\n\t", extractor.enabledFeatures)


Enabled filters:
	 {'LoG': {}, 'Original': {}}

Enabled features:
	 {'firstorder': []}

Enabled features:
	 {'glcm': ['Autocorrelation', 'Homogeneity1', 'SumSquares'], 'firstorder': []}


##### Method 3, using a parameter file

In [7]:
# Instantiate the extractor
extractor = featureextractor.RadiomicsFeaturesExtractor(paramPath)

print("Extraction parameters:\n\t", extractor.kwargs)
print("Enabled filters:\n\t", extractor.inputImages)
print("Enabled features:\n\t", extractor.enabledFeatures)

Extraction parameters:
	 {'verbose': True, 'binWidth': 25, 'resampledPixelSpacing': None, 'weightingNorm': None, 'interpolator': 'sitkBSpline', 'padDistance': 5, 'label': 1}
Enabled filters:
	 {'Original': {}}
Enabled features:
	 {'glcm': None, 'glrlm': None, 'glszm': None, 'shape': None, 'firstorder': []}


### Extract features

Now that we have our extractor set up with the correct parameters, we can start extracting features:

In [8]:
result = extractor.execute(imagePath, labelPath)

FAILED: Traceback (most recent call last):
  File "/Users/mader/anaconda/lib/python3.5/site-packages/pyradiomics-1.0.1.post5.dev0+g72ac3cb-py3.5.egg/radiomics/base.py", line 73, in calculateFeatures
    self.featureValues[feature] = eval(call)
  File "<string>", line 1, in <module>
  File "/Users/mader/anaconda/lib/python3.5/site-packages/pyradiomics-1.0.1.post5.dev0+g72ac3cb-py3.5.egg/radiomics/shape.py", line 228, in getMaximum2DDiameterColumnFeatureValue
    return self._getMaximum2Ddiameter(1)
  File "/Users/mader/anaconda/lib/python3.5/site-packages/pyradiomics-1.0.1.post5.dev0+g72ac3cb-py3.5.egg/radiomics/shape.py", line 126, in _getMaximum2Ddiameter
    for i in numpy.unique(a[:, dim]):
IndexError: too many indices for array

FAILED: Traceback (most recent call last):
  File "/Users/mader/anaconda/lib/python3.5/site-packages/pyradiomics-1.0.1.post5.dev0+g72ac3cb-py3.5.egg/radiomics/base.py", line 73, in calculateFeatures
    self.featureValues[feature] = eval(call)
  File "<stri

		Computing shape


calculate GLCM: 100%|██████████| 33/33 [00:01<00:00, 32.69it/s]


		Computing glcm


calculate GLSZM:  27%|██▋       | 9/33 [00:00<00:00, 89.69it/s]

		Computing glrlm


calculate GLSZM: 100%|██████████| 33/33 [00:00<00:00, 72.79it/s]

		Computing glszm
		Computing firstorder


In [9]:
print("Result type:", type(result))  # result is returned in a Python ordered dictionary
print("")
print("Calculated features")
for key, value in result.items():
    print("\t", key, ":", value)

Result type: <class 'collections.OrderedDict'>

Calculated features
	 general_info_BoundingBox : (162; 84; 11; 47; 70; 7)
	 general_info_GeneralSettings : {'verbose': True; 'binWidth': 25; 'resampledPixelSpacing': None; 'weightingNorm': None; 'interpolator': 'sitkBSpline'; 'padDistance': 5; 'label': 1}
	 general_info_ImageHash : 5c9ce3ca174f0f8324aa4d277e0fef82dc5ac566
	 general_info_ImageSpacing : (0.7812499999999999; 0.7812499999999999; 6.499999999999998)
	 general_info_InputImages : {'Original': {}}
	 general_info_MaskHash : 9dc2c3137b31fd872997d92c9a92d5178126d9d3
	 general_info_Version : v1.0.1.post5.dev0+g72ac3cb
	 general_info_VolumeNum : 2
	 general_info_VoxelNum : 4137
	 original_shape_SurfaceArea : 6438.82160378
	 original_shape_Sphericity : 0.485061744222
	 original_shape_Maximum2DDiameterColumn : nan
	 original_shape_Roundness : 0.6146906661500379
	 original_shape_Compactness1 : 26.7546787215
	 original_shape_SurfaceVolumeRatio : 0.392308261863
	 original_shape_Maximum2DDia